In [ ]:
import requests
import random
import pandas as pd
import os
from pathlib import Path
import win32com.client as win32
import win32com
import time
from datetime import datetime, timedelta
import pyotp
import pandas as pd
from SmartApi import SmartConnect
from logzero import logger
import requests

class OptionChainUpdaterCOM:
    def __init__(self, file_path: str):
        self.file_path = Path(file_path)
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
            "Accept": "application/json, text/plain, */*",
            "Referer": "https://www.nseindia.com/option-chain"
        }

    def fetch_data(self):
        random_suffix = random.randint(100000, 999999)
        oc_url = f"https://www.nseindia.com/api/option-chain-indices?symbol=NIFTY&c={random_suffix}"

        session = requests.Session()
        session.get("https://www.nseindia.com/option-chain", headers=self.headers)
        response = session.get(oc_url, headers=self.headers)

        if response.status_code != 200:
            raise Exception(f"Failed to fetch data. Status code: {response.status_code}")

        full_data = response.json()
        return full_data['records']['data'], full_data['records']['expiryDates'][:1], full_data['records']['timestamp']

    def prepare_dataframe(self, data, top_expiries):
        rows = []
        for item in data:
            expiry = item.get("expiryDate")
            if expiry not in top_expiries:
                continue

            row = {
                "EXPIRY DATE": expiry,
                "STRIKE": item.get("strikePrice")
            }

            ce = item.get("CE", {})
            row.update({
                "CE OI": ce.get("openInterest"),
                "CE CHNG IN OI": ce.get("changeinOpenInterest"),
                "CE VOLUME": ce.get("totalTradedVolume"),
                "CE IV": ce.get("impliedVolatility"),
                "CE LTP": ce.get("lastPrice"),
                "CE CHNG": ce.get("change"),
                "CE BID QTY": ce.get("bidQty"),
                "CE BID": ce.get("bidprice"),
                "CE ASK": ce.get("askPrice"),
                "CE ASK QTY": ce.get("askQty"),
                "CE TOTAL BUY": ce.get("totalBuyQuantity"),
                "CE TOTAL SELL": ce.get("totalSellQuantity"),
            })

            pe = item.get("PE", {})
            row.update({
                "PE OI": pe.get("openInterest"),
                "PE CHNG IN OI": pe.get("changeinOpenInterest"),
                "PE VOLUME": pe.get("totalTradedVolume"),
                "PE IV": pe.get("impliedVolatility"),
                "PE LTP": pe.get("lastPrice"),
                "PE CHNG": pe.get("change"),
                "PE BID QTY": pe.get("bidQty"),
                "PE BID": pe.get("bidprice"),
                "PE ASK": pe.get("askPrice"),
                "PE ASK QTY": pe.get("askQty"),
                "PE TOTAL BUY": pe.get("totalBuyQuantity"),
                "PE TOTAL SELL": pe.get("totalSellQuantity"),
            })

            rows.append(row)

        df = pd.DataFrame(rows)
        df.sort_values(by=["EXPIRY DATE", "STRIKE"], inplace=True)
        return df

    def write_using_com(self, df: pd.DataFrame, timestamp: str):
        excel = win32.Dispatch("Excel.Application")
        excel.Visible = True

        # Check if workbook is already open
        wb = None
        for open_wb in excel.Workbooks:
            if open_wb.FullName.lower() == str(self.file_path).lower():
                wb = open_wb
                break

        if wb is None:
            wb = excel.Workbooks.Open(str(self.file_path))

        # Ensure a second worksheet exists
        if wb.Sheets.Count < 2:
            wb.Sheets.Add(After=wb.Sheets(wb.Sheets.Count))  # Add new sheet after the last one

        # Use the second worksheet
        ws = wb.Worksheets(2)
        ws.Name = "Nifty option chain"

        # Clear the worksheet contents
        ws.Cells.Clear()

        # Row 1 - Timestamp
        ws.Cells(1, 1).Value = f"Data fetched at: {timestamp}"

        headers = df.columns.tolist()

        # Assuming first 3 columns are general, then CALLS (next 10), then PUTS (last 10)
        num_general = 3
        num_calls = 10
        num_puts = len(headers) - num_general - num_calls

        # Row 2 - Group header (CALLS, PUTS)
        for i in range(num_general):
            cell = ws.Cells(2, i + 1)
            cell.Value = ""  # No group for general columns
            cell.Font.Bold = True
            cell.Interior.Color = 65535  # Yellow highlight

        if num_calls > 0:
            ws.Range(
                ws.Cells(2, num_general + 1),
                ws.Cells(2, num_general + num_calls)
            ).Merge()
            call_cell = ws.Cells(2, num_general + 1)
            call_cell.Value = "CALLS"
            call_cell.HorizontalAlignment = -4108  # Center
            call_cell.Font.Bold = True
            call_cell.Interior.Color = 65535  # Yellow highlight

        if num_puts > 0:
            ws.Range(
                ws.Cells(2, num_general + num_calls + 1),
                ws.Cells(2, num_general + num_calls + num_puts)
            ).Merge()
            put_cell = ws.Cells(2, num_general + num_calls + 1)
            put_cell.Value = "PUTS"
            put_cell.HorizontalAlignment = -4108  # Center
            put_cell.Font.Bold = True
            put_cell.Interior.Color = 65535  # Yellow highlight


        # Row 3 - Actual headers
        for col_index, header in enumerate(headers, start=1):
            cell = ws.Cells(3, col_index)
            cell.Value = header
            cell.Font.Bold = True  # Make the header bold

        # Data rows start at row 4
        for row_index, row in enumerate(df.itertuples(index=False), start=4):
            for col_index, value in enumerate(row, start=1):
                ws.Cells(row_index, col_index).Value = value

        # Apply filter
        last_col = df.shape[1]
        last_row = df.shape[0] + 3
        try:
            if df.shape[0] > 0 and df.shape[1] > 0 and last_row >= 3 and last_col >= 1:
                print("12.2")
                ws.Range(ws.Cells(3, 1), ws.Cells(last_row, last_col)).AutoFilter()
                print("12.3")
            else:
                print("No data to filter, skipping AutoFilter.")
        except Exception as e:
            print(f"AutoFilter error: {e}")

    def run(self):
        print("Fetching Bank Nifty option chain data...")
        data, expiries, timestamp = self.fetch_data()
        print(f"Data fetched at: {timestamp}")
        df = self.prepare_dataframe(data, expiries)
        print("Preparing DataFrame...")
        self.write_using_com(df, timestamp)
        print(f"Nifty options updated in Excel at {timestamp}")

class BankOptionChainUpdaterCOM:
    def __init__(self, file_path: str):
        self.file_path = Path(file_path)
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
            "Accept": "application/json, text/plain, */*",
            "Referer": "https://www.nseindia.com/option-chain"
        }

    def fetch_data(self):
        random_suffix = random.randint(100000, 999999)
        oc_url = f"https://www.nseindia.com/api/option-chain-indices?symbol=BANKNIFTY&c={random_suffix}"

        session = requests.Session()
        session.get("https://www.nseindia.com/option-chain", headers=self.headers)
        response = session.get(oc_url, headers=self.headers)

        if response.status_code != 200:
            raise Exception(f"Failed to fetch data. Status code: {response.status_code}")

        full_data = response.json()
        return full_data['records']['data'], full_data['records']['expiryDates'][:1], full_data['records']['timestamp']

    def prepare_dataframe(self, data, top_expiries):
        rows = []
        for item in data:
            expiry = item.get("expiryDate")
            if expiry not in top_expiries:
                continue

            row = {
                "EXPIRY DATE": expiry,
                "STRIKE": item.get("strikePrice")
            }

            ce = item.get("CE", {})
            row.update({
                "CE OI": ce.get("openInterest"),
                "CE CHNG IN OI": ce.get("changeinOpenInterest"),
                "CE VOLUME": ce.get("totalTradedVolume"),
                "CE IV": ce.get("impliedVolatility"),
                "CE LTP": ce.get("lastPrice"),
                "CE CHNG": ce.get("change"),
                "CE BID QTY": ce.get("bidQty"),
                "CE BID": ce.get("bidprice"),
                "CE ASK": ce.get("askPrice"),
                "CE ASK QTY": ce.get("askQty"),
                "CE TOTAL BUY": ce.get("totalBuyQuantity"),
                "CE TOTAL SELL": ce.get("totalSellQuantity"),
            })

            pe = item.get("PE", {})
            row.update({
                "PE OI": pe.get("openInterest"),
                "PE CHNG IN OI": pe.get("changeinOpenInterest"),
                "PE VOLUME": pe.get("totalTradedVolume"),
                "PE IV": pe.get("impliedVolatility"),
                "PE LTP": pe.get("lastPrice"),
                "PE CHNG": pe.get("change"),
                "PE BID QTY": pe.get("bidQty"),
                "PE BID": pe.get("bidprice"),
                "PE ASK": pe.get("askPrice"),
                "PE ASK QTY": pe.get("askQty"),
                "PE TOTAL BUY": pe.get("totalBuyQuantity"),
                "PE TOTAL SELL": pe.get("totalSellQuantity"),
            })

            rows.append(row)

        df = pd.DataFrame(rows)
        df.sort_values(by=["EXPIRY DATE", "STRIKE"], inplace=True)
        return df

    def write_using_com(self, df: pd.DataFrame, timestamp: str):
        excel = win32.Dispatch("Excel.Application")
        excel.Visible = True

        try: 
            # Check if workbook is already open
            wb = None
            for open_wb in excel.Workbooks:
                if open_wb.FullName.lower() == str(self.file_path).lower():
                    wb = open_wb
                    break

            if wb is None:
                wb = excel.Workbooks.Open(str(self.file_path))
        
            # Ensure a second worksheet exists
            if wb.Sheets.Count < 3:
                wb.Sheets.Add(After=wb.Sheets(wb.Sheets.Count))  # Add new sheet after the last one
            
            # Use the second worksheet
            ws = wb.Worksheets(3)
            ws.Name = "Bank_Nifty option chain"
            
            # Clear the worksheet contents
            ws.Cells.Clear()
            
            # Row 1 - Timestamp
            ws.Cells(1, 1).Value = f"Data fetched at: {timestamp}"
        
            headers = df.columns.tolist()

            # Assuming first 3 columns are general, then CALLS (next 10), then PUTS (last 10)
            num_general = 3
            num_calls = 10
            num_puts = len(headers) - num_general - num_calls
        
            # Row 2 - Group header (CALLS, PUTS)
            for i in range(num_general):
                cell = ws.Cells(2, i + 1)
                cell.Value = ""  # No group for general columns
                cell.Font.Bold = True
                cell.Interior.Color = 65535  # Yellow highlight
        
            if num_calls > 0:
                ws.Range(
                    ws.Cells(2, num_general + 1),
                    ws.Cells(2, num_general + num_calls)
                ).Merge()
                call_cell = ws.Cells(2, num_general + 1)
                call_cell.Value = "CALLS"
                call_cell.HorizontalAlignment = -4108  # Center
                call_cell.Font.Bold = True
                call_cell.Interior.Color = 65535  # Yellow highlight

            if num_puts > 0:
                ws.Range(
                    ws.Cells(2, num_general + num_calls + 1),
                    ws.Cells(2, num_general + num_calls + num_puts)
                ).Merge()
                put_cell = ws.Cells(2, num_general + num_calls + 1)
                put_cell.Value = "PUTS"
                put_cell.HorizontalAlignment = -4108  # Center
                put_cell.Font.Bold = True
                put_cell.Interior.Color = 65535  # Yellow highlight

            # Row 3 - Actual headers
            for col_index, header in enumerate(headers, start=1):
                cell = ws.Cells(3, col_index)
                cell.Value = header
                cell.Font.Bold = True  # Make the header bold
        
            # Data rows start at row 4
            for row_index, row in enumerate(df.itertuples(index=False), start=4):
                for col_index, value in enumerate(row, start=1):
                    ws.Cells(row_index, col_index).Value = value
        
            # Apply filter
            last_col = df.shape[1]
            last_row = df.shape[0] + 3
            
            try:
                if df.shape[0] > 0 and df.shape[1] > 0 and last_row >= 3 and last_col >= 1:
                    ws.Range(ws.Cells(3, 1), ws.Cells(last_row, last_col)).AutoFilter()
                else:
                    print("No data to filter, skipping AutoFilter.")
            except Exception as e:
                print(f"AutoFilter error: {e}")
            
            wb.Save()
        
        except Exception as e:
            print(f"Error during update: {e}")


    def run(self):
        print("Fetching Bank Nifty option chain data...")
        data, expiries, timestamp = self.fetch_data()
        print(f"Data fetched successfully at {timestamp}. Preparing DataFrame...")
        df = self.prepare_dataframe(data, expiries)
        print("DataFrame prepared. Writing to Excel...")
        self.write_using_com(df, timestamp)
        print(f"Bank Nifty option chain data updated successfully at {timestamp}.")

In [ ]:
file_path = r"C:\Users\jaiad\Pro T\Final_draft1\banknifty_dashboard.xlsx"

banknifty_args = {
    "api_key": '3tA3noq3',
    "username": 'AAAF620196',
    "pwd": '0054',
    "token": 'YS4MOR7OUZEHGNV3VFOQ5TVMRE',
    "file_path": file_path,
    "offset_rows": 15
}
indices_args = {
    "api_key": '3tA3noq3',
    "username": 'AAAF620196',
    "pwd": '0054',
    "token": 'YS4MOR7OUZEHGNV3VFOQ5TVMRE',
    "file_path": file_path
}


class DashboardIntegrator:
    def __init__(
        self,
        banknifty_args,
        indices_args,
        banknifty_interval=30
    ):
        # self.banknifty_dashboard = BankNiftyDashboard(**banknifty_args)
        # self.indices_dashboard = IndicesDashboard(**indices_args)
        # self.indices_dashboard.clear_dashboard()
        # self.indices_dashboard.setup_excel_structure()
        self.banknifty_interval = banknifty_interval
        self.niftyupdater = OptionChainUpdaterCOM(file_path=file_path)
        self.bankniftyupdater = BankOptionChainUpdaterCOM(file_path=file_path)

    def run(self):

        try:
            while True:
                try:
                    # print("Updating Indices Dashboard...")
                    # df = self.indices_dashboard.fetch_and_process_data()
                    # df = self.indices_dashboard.calculate_sentiment(df)
                    # self.indices_dashboard.update_excel(df)
                    # print("Updating BankNifty Dashboard...")
                    # self.banknifty_dashboard.update_excel()
                    # print(f"Both dashboards updated at {datetime.now().strftime('%H:%M:%S')}")
                    self.bankniftyupdater.run()
                    # self.niftyupdater.run()
                except Exception as e:
                    print(f"Error during update: {str(e)}")
                time.sleep(self.banknifty_interval)
        except KeyboardInterrupt:
            print("Stopped by user.")
        except Exception as e:
            print("Unexpected error:", e)

integrator = DashboardIntegrator(banknifty_args, indices_args, banknifty_interval=0)
integrator.run()

Fetching Bank Nifty option chain data...


Data fetched successfully at 01-Aug-2025 15:30:00. Preparing DataFrame...
DataFrame prepared. Writing to Excel...
Error during update: (-2146777998, 'OLE error 0x800ac472', None, None)
Bank Nifty option chain data updated successfully at 01-Aug-2025 15:30:00.
Fetching Bank Nifty option chain data...
Data fetched successfully at 01-Aug-2025 15:30:00. Preparing DataFrame...
DataFrame prepared. Writing to Excel...
Error during update: (-2146777998, 'OLE error 0x800ac472', None, None)
Bank Nifty option chain data updated successfully at 01-Aug-2025 15:30:00.
Fetching Bank Nifty option chain data...
Error during update: HTTPSConnectionPool(host='www.nseindia.com', port=443): Max retries exceeded with url: /option-chain (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001C9C968DEE0>: Failed to resolve 'www.nseindia.com' ([Errno 11001] getaddrinfo failed)"))
Fetching Bank Nifty option chain data...
Data fetched successfully at 01-Aug-2025 15:30:00. Preparing 

# Running Trials

In [ ]:
import requests
import random
import pandas as pd
import os
from pathlib import Path
import win32com.client as win32
import win32com
import time
from datetime import datetime, timedelta
import pyotp
import pandas as pd
from SmartApi import SmartConnect
from logzero import logger
import requests
import pythoncom

class BankOptionChainUpdaterCOM:
    def __init__(self, file_path: str):
        self.file_path = Path(file_path)
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
            "Accept": "application/json, text/plain, */*",
            "Referer": "https://www.nseindia.com/option-chain"
        }

    def fetch_data(self):
        random_suffix = random.randint(100000, 999999)
        oc_url = f"https://www.nseindia.com/api/option-chain-indices?symbol=BANKNIFTY&c={random_suffix}"

        session = requests.Session()
        session.get("https://www.nseindia.com/option-chain", headers=self.headers)
        response = session.get(oc_url, headers=self.headers)

        if response.status_code != 200:
            raise Exception(f"Failed to fetch data. Status code: {response.status_code}")

        full_data = response.json()
        return full_data['records']['data'], full_data['records']['expiryDates'][:1], full_data['records']['timestamp']

    def prepare_dataframe(self, data, top_expiries):
        rows = []
        for item in data:
            expiry = item.get("expiryDate")
            if expiry not in top_expiries:
                continue

            row = {
                "EXPIRY DATE": expiry,
                "STRIKE": item.get("strikePrice")
            }

            ce = item.get("CE", {})
            row.update({
                "CE OI": ce.get("openInterest"),
                "CE CHNG IN OI": ce.get("changeinOpenInterest"),
                "CE VOLUME": ce.get("totalTradedVolume"),
                "CE IV": ce.get("impliedVolatility"),
                "CE LTP": ce.get("lastPrice"),
                "CE CHNG": ce.get("change"),
                "CE BID QTY": ce.get("bidQty"),
                "CE BID": ce.get("bidprice"),
                "CE ASK": ce.get("askPrice"),
                "CE ASK QTY": ce.get("askQty"),
                "CE TOTAL BUY": ce.get("totalBuyQuantity"),
                "CE TOTAL SELL": ce.get("totalSellQuantity"),
            })

            pe = item.get("PE", {})
            row.update({
                "PE OI": pe.get("openInterest"),
                "PE CHNG IN OI": pe.get("changeinOpenInterest"),
                "PE VOLUME": pe.get("totalTradedVolume"),
                "PE IV": pe.get("impliedVolatility"),
                "PE LTP": pe.get("lastPrice"),
                "PE CHNG": pe.get("change"),
                "PE BID QTY": pe.get("bidQty"),
                "PE BID": pe.get("bidprice"),
                "PE ASK": pe.get("askPrice"),
                "PE ASK QTY": pe.get("askQty"),
                "PE TOTAL BUY": pe.get("totalBuyQuantity"),
                "PE TOTAL SELL": pe.get("totalSellQuantity"),
            })

            rows.append(row)

        df = pd.DataFrame(rows)
        df.sort_values(by=["EXPIRY DATE", "STRIKE"], inplace=True)
        return df

    def run(self):
        print("Fetching Bank Nifty option chain data...")
        data, expiries, timestamp = self.fetch_data()
        print(f"Data fetched successfully at {timestamp}. Preparing DataFrame...")
        df = self.prepare_dataframe(data, expiries)
        print("DataFrame prepared. Writing to Excel...")
        self.write_using_com(df, timestamp)

In [55]:
import threading
import win32com.client as win32
import pandas as pd
import pythoncom
import time

def write_using_com(self, df: pd.DataFrame, timestamp: str):
    def safe_set_attr(obj_getter, attr, value, retries=5, delay=0.5):
        for i in range(retries):
            try:
                obj = obj_getter()
                setattr(obj, attr, value)
                return
            except Exception as e:
                if '0x800ac472' in str(e):
                    time.sleep(delay)
                else:
                    raise
        print(f"Failed to set {attr} after {retries} attempts.")

    pythoncom.CoInitialize()
    excel = win32.Dispatch("Excel.Application")
    excel.Visible = True

    # Temporarily disable screen updating and alert
    excel.DisplayAlerts = False

    try:
        # Check if workbook is already open
        wb = None
        for open_wb in excel.Workbooks:
            if open_wb.FullName.lower() == str(self.file_path).lower():
                wb = open_wb
                break

        if wb is None:
            wb = excel.Workbooks.Open(str(self.file_path))

        # Ensure at least 3 worksheets
        while wb.Sheets.Count < 3:
            wb.Sheets.Add(After=wb.Sheets(wb.Sheets.Count))

        ws = wb.Worksheets(3)
        ws.Name = "Bank_Nifty option chain"

        try:
            if ws.ProtectContents:
                ws.Unprotect()
        except Exception as e:
            print(f"Warning: Couldn't unprotect sheet: {e}")

        ws.Cells.Clear()

        safe_set_attr(lambda: ws.Cells(1, 1), "Value", f"Data fetched at: {timestamp}")

        headers = df.columns.tolist()
        num_general = 3
        num_calls = 10
        num_puts = len(headers) - num_general - num_calls

        for i in range(num_general):
            cell = ws.Cells(2, i + 1)
            safe_set_attr(lambda c=cell: c, "Value", "")
            safe_set_attr(lambda c=cell: c.Font, "Bold", True)
            safe_set_attr(lambda c=cell: c.Interior, "Color", 65535)

        if num_calls > 0:
            ws.Range(
                ws.Cells(2, num_general + 1),
                ws.Cells(2, num_general + num_calls)
            ).Merge()
            call_cell = ws.Cells(2, num_general + 1)
            safe_set_attr(lambda: call_cell, "Value", "CALLS")
            safe_set_attr(lambda: call_cell, "HorizontalAlignment", -4108)
            safe_set_attr(lambda: call_cell.Font, "Bold", True)
            safe_set_attr(lambda: call_cell.Interior, "Color", 65535)

        if num_puts > 0:
            ws.Range(
                ws.Cells(2, num_general + num_calls + 1),
                ws.Cells(2, num_general + num_calls + num_puts)
            ).Merge()
            put_cell = ws.Cells(2, num_general + num_calls + 1)
            safe_set_attr(lambda: put_cell, "Value", "PUTS")
            safe_set_attr(lambda: put_cell, "HorizontalAlignment", -4108)
            safe_set_attr(lambda: put_cell.Font, "Bold", True)
            safe_set_attr(lambda: put_cell.Interior, "Color", 65535)

        for col_index, header in enumerate(headers, start=1):
            cell = ws.Cells(3, col_index)
            safe_set_attr(lambda c=cell: c, "Value", header)
            safe_set_attr(lambda c=cell: c.Font, "Bold", True)

        for row_index, row in enumerate(df.itertuples(index=False), start=4):
            for col_index, value in enumerate(row, start=1):
                safe_set_attr(lambda r=row_index, c=col_index: ws.Cells(r, c), "Value", value)

        last_col = df.shape[1]
        last_row = df.shape[0] + 3

        try:
            if df.shape[0] > 0 and df.shape[1] > 0 and last_row >= 3 and last_col >= 1:
                ws.Range(ws.Cells(3, 1), ws.Cells(last_row, last_col)).AutoFilter()
            else:
                print("No data to filter, skipping AutoFilter.")
        except Exception as e:
            print(f"AutoFilter error: {e}")

        wb.Save()

    except Exception as e:
        print(f"Error during update: {e}")

    finally:
        # Restore Excel settings
        excel.DisplayAlerts = True
        pythoncom.CoUninitialize()
        print(f"Bank Nifty option chain data updated successfully at {timestamp}.")

BankOptionChainUpdaterCOM.write_using_com = write_using_com

In [56]:
file_path = r"C:\Users\jaiad\Pro T\Final_draft1\interactive_trials.xls"

banknifty_args = {
    "api_key": '3tA3noq3',
    "username": 'AAAF620196',
    "pwd": '0054',
    "token": 'YS4MOR7OUZEHGNV3VFOQ5TVMRE',
    "file_path": file_path,
    "offset_rows": 15
}
indices_args = {
    "api_key": '3tA3noq3',
    "username": 'AAAF620196',
    "pwd": '0054',
    "token": 'YS4MOR7OUZEHGNV3VFOQ5TVMRE',
    "file_path": file_path
}


class DashboardIntegrator:
    def __init__(
        self,
        banknifty_args,
        indices_args,
        banknifty_interval=30
    ):
        # self.banknifty_dashboard = BankNiftyDashboard(**banknifty_args)
        # self.indices_dashboard = IndicesDashboard(**indices_args)
        # self.indices_dashboard.clear_dashboard()
        # self.indices_dashboard.setup_excel_structure()
        self.banknifty_interval = banknifty_interval
        # self.niftyupdater = OptionChainUpdaterCOM(file_path=file_path)
        self.bankniftyupdater = BankOptionChainUpdaterCOM(file_path=file_path)

    def run(self):

        try:
            while True:
                try:
                    # print("Updating Indices Dashboard...")
                    # df = self.indices_dashboard.fetch_and_process_data()
                    # df = self.indices_dashboard.calculate_sentiment(df)
                    # self.indices_dashboard.update_excel(df)
                    # print("Updating BankNifty Dashboard...")
                    # self.banknifty_dashboard.update_excel()
                    # print(f"Both dashboards updated at {datetime.now().strftime('%H:%M:%S')}")
                    self.bankniftyupdater.run()
                    # self.niftyupdater.run()
                except Exception as e:
                    print(f"Error during update: {str(e)}")
                time.sleep(self.banknifty_interval)
        except KeyboardInterrupt:
            print("Stopped by user.")
        except Exception as e:
            print("Unexpected error:", e)

integrator = DashboardIntegrator(banknifty_args, indices_args, banknifty_interval=10)
integrator.run()

Fetching Bank Nifty option chain data...
Data fetched successfully at 01-Aug-2025 15:30:00. Preparing DataFrame...
DataFrame prepared. Writing to Excel...
Error during update: (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2146777998), None)
Error during update: (-2146777998, 'OLE error 0x800ac472', None, None)
Fetching Bank Nifty option chain data...
Data fetched successfully at 01-Aug-2025 15:30:00. Preparing DataFrame...
DataFrame prepared. Writing to Excel...
Error during update: (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2146777998), None)
Error during update: (-2146777998, 'OLE error 0x800ac472', None, None)
Fetching Bank Nifty option chain data...
Data fetched successfully at 01-Aug-2025 15:30:00. Preparing DataFrame...
DataFrame prepared. Writing to Excel...
Error during update: (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2146777998), None)
Error during update: (-2146777998, 'OLE error 0x800ac472', None, None)
Fetching Bank

# Check Point 1

In [57]:
import requests
import random
import pandas as pd
import os
from pathlib import Path
import win32com.client as win32
import win32com
import time
from datetime import datetime, timedelta
import pyotp
import pandas as pd
from SmartApi import SmartConnect
from logzero import logger
import requests
import pythoncom

class BankOptionChainUpdaterCOM:
    def __init__(self, file_path: str):
        self.file_path = Path(file_path)
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
            "Accept": "application/json, text/plain, */*",
            "Referer": "https://www.nseindia.com/option-chain"
        }

    def fetch_data(self):
        random_suffix = random.randint(100000, 999999)
        oc_url = f"https://www.nseindia.com/api/option-chain-indices?symbol=BANKNIFTY&c={random_suffix}"

        session = requests.Session()
        session.get("https://www.nseindia.com/option-chain", headers=self.headers)
        response = session.get(oc_url, headers=self.headers)

        if response.status_code != 200:
            raise Exception(f"Failed to fetch data. Status code: {response.status_code}")

        full_data = response.json()
        return full_data['records']['data'], full_data['records']['expiryDates'][:1], full_data['records']['timestamp']

    def prepare_dataframe(self, data, top_expiries):
        rows = []
        for item in data:
            expiry = item.get("expiryDate")
            if expiry not in top_expiries:
                continue

            row = {
                "EXPIRY DATE": expiry,
                "STRIKE": item.get("strikePrice")
            }

            ce = item.get("CE", {})
            row.update({
                "CE OI": ce.get("openInterest"),
                "CE CHNG IN OI": ce.get("changeinOpenInterest"),
                "CE VOLUME": ce.get("totalTradedVolume"),
                "CE IV": ce.get("impliedVolatility"),
                "CE LTP": ce.get("lastPrice"),
                "CE CHNG": ce.get("change"),
                "CE BID QTY": ce.get("bidQty"),
                "CE BID": ce.get("bidprice"),
                "CE ASK": ce.get("askPrice"),
                "CE ASK QTY": ce.get("askQty"),
                "CE TOTAL BUY": ce.get("totalBuyQuantity"),
                "CE TOTAL SELL": ce.get("totalSellQuantity"),
            })

            pe = item.get("PE", {})
            row.update({
                "PE OI": pe.get("openInterest"),
                "PE CHNG IN OI": pe.get("changeinOpenInterest"),
                "PE VOLUME": pe.get("totalTradedVolume"),
                "PE IV": pe.get("impliedVolatility"),
                "PE LTP": pe.get("lastPrice"),
                "PE CHNG": pe.get("change"),
                "PE BID QTY": pe.get("bidQty"),
                "PE BID": pe.get("bidprice"),
                "PE ASK": pe.get("askPrice"),
                "PE ASK QTY": pe.get("askQty"),
                "PE TOTAL BUY": pe.get("totalBuyQuantity"),
                "PE TOTAL SELL": pe.get("totalSellQuantity"),
            })

            rows.append(row)

        df = pd.DataFrame(rows)
        df.sort_values(by=["EXPIRY DATE", "STRIKE"], inplace=True)
        return df
    
    def write_using_com(self, df: pd.DataFrame, timestamp: str):
        def safe_set_attr(obj_getter, attr, value, retries=5, delay=0.5):
            for i in range(retries):
                try:
                    obj = obj_getter()
                    setattr(obj, attr, value)
                    return
                except Exception as e:
                    if '0x800ac472' in str(e):
                        time.sleep(delay)
                    else:
                        raise
            print(f"Failed to set {attr} after {retries} attempts.")

        pythoncom.CoInitialize()
        excel = win32.Dispatch("Excel.Application")
        excel.Visible = True

        # Temporarily disable screen updating and alert
        excel.DisplayAlerts = False

        try:
            # Check if workbook is already open
            wb = None
            for open_wb in excel.Workbooks:
                if open_wb.FullName.lower() == str(self.file_path).lower():
                    wb = open_wb
                    break

            if wb is None:
                wb = excel.Workbooks.Open(str(self.file_path))

            # Ensure at least 3 worksheets
            while wb.Sheets.Count < 3:
                wb.Sheets.Add(After=wb.Sheets(wb.Sheets.Count))

            ws = wb.Worksheets(3)
            ws.Name = "Bank_Nifty option chain"

            try:
                if ws.ProtectContents:
                    ws.Unprotect()
            except Exception as e:
                print(f"Warning: Couldn't unprotect sheet: {e}")

            ws.Cells.Clear()

            safe_set_attr(lambda: ws.Cells(1, 1), "Value", f"Data fetched at: {timestamp}")

            headers = df.columns.tolist()
            num_general = 3
            num_calls = 10
            num_puts = len(headers) - num_general - num_calls

            for i in range(num_general):
                cell = ws.Cells(2, i + 1)
                safe_set_attr(lambda c=cell: c, "Value", "")
                safe_set_attr(lambda c=cell: c.Font, "Bold", True)
                safe_set_attr(lambda c=cell: c.Interior, "Color", 65535)

            if num_calls > 0:
                ws.Range(
                    ws.Cells(2, num_general + 1),
                    ws.Cells(2, num_general + num_calls)
                ).Merge()
                call_cell = ws.Cells(2, num_general + 1)
                safe_set_attr(lambda: call_cell, "Value", "CALLS")
                safe_set_attr(lambda: call_cell, "HorizontalAlignment", -4108)
                safe_set_attr(lambda: call_cell.Font, "Bold", True)
                safe_set_attr(lambda: call_cell.Interior, "Color", 65535)

            if num_puts > 0:
                ws.Range(
                    ws.Cells(2, num_general + num_calls + 1),
                    ws.Cells(2, num_general + num_calls + num_puts)
                ).Merge()
                put_cell = ws.Cells(2, num_general + num_calls + 1)
                safe_set_attr(lambda: put_cell, "Value", "PUTS")
                safe_set_attr(lambda: put_cell, "HorizontalAlignment", -4108)
                safe_set_attr(lambda: put_cell.Font, "Bold", True)
                safe_set_attr(lambda: put_cell.Interior, "Color", 65535)

            for col_index, header in enumerate(headers, start=1):
                cell = ws.Cells(3, col_index)
                safe_set_attr(lambda c=cell: c, "Value", header)
                safe_set_attr(lambda c=cell: c.Font, "Bold", True)

            for row_index, row in enumerate(df.itertuples(index=False), start=4):
                for col_index, value in enumerate(row, start=1):
                    safe_set_attr(lambda r=row_index, c=col_index: ws.Cells(r, c), "Value", value)

            last_col = df.shape[1]
            last_row = df.shape[0] + 3

            try:
                if df.shape[0] > 0 and df.shape[1] > 0 and last_row >= 3 and last_col >= 1:
                    ws.Range(ws.Cells(3, 1), ws.Cells(last_row, last_col)).AutoFilter()
                else:
                    print("No data to filter, skipping AutoFilter.")
            except Exception as e:
                print(f"AutoFilter error: {e}")

            wb.Save()

        except Exception as e:
            print(f"Error during update: {e}")

        finally:
            # Restore Excel settings
            excel.DisplayAlerts = True
            pythoncom.CoUninitialize()
            print(f"Bank Nifty option chain data updated successfully at {timestamp}.")

    def run(self):
        print("Fetching Bank Nifty option chain data...")
        data, expiries, timestamp = self.fetch_data()
        print(f"Data fetched successfully at {timestamp}. Preparing DataFrame...")
        df = self.prepare_dataframe(data, expiries)
        print("DataFrame prepared. Writing to Excel...")
        self.write_using_com(df, timestamp)

In [58]:
file_path = r"C:\Users\jaiad\Pro T\Final_draft1\interactive_trials.xls"

banknifty_args = {
    "api_key": '3tA3noq3',
    "username": 'AAAF620196',
    "pwd": '0054',
    "token": 'YS4MOR7OUZEHGNV3VFOQ5TVMRE',
    "file_path": file_path,
    "offset_rows": 15
}
indices_args = {
    "api_key": '3tA3noq3',
    "username": 'AAAF620196',
    "pwd": '0054',
    "token": 'YS4MOR7OUZEHGNV3VFOQ5TVMRE',
    "file_path": file_path
}


class DashboardIntegrator:
    def __init__(
        self,
        banknifty_args,
        indices_args,
        banknifty_interval=30
    ):
        # self.banknifty_dashboard = BankNiftyDashboard(**banknifty_args)
        # self.indices_dashboard = IndicesDashboard(**indices_args)
        # self.indices_dashboard.clear_dashboard()
        # self.indices_dashboard.setup_excel_structure()
        self.banknifty_interval = banknifty_interval
        # self.niftyupdater = OptionChainUpdaterCOM(file_path=file_path)
        self.bankniftyupdater = BankOptionChainUpdaterCOM(file_path=file_path)

    def run(self):

        try:
            while True:
                try:
                    # print("Updating Indices Dashboard...")
                    # df = self.indices_dashboard.fetch_and_process_data()
                    # df = self.indices_dashboard.calculate_sentiment(df)
                    # self.indices_dashboard.update_excel(df)
                    # print("Updating BankNifty Dashboard...")
                    # self.banknifty_dashboard.update_excel()
                    # print(f"Both dashboards updated at {datetime.now().strftime('%H:%M:%S')}")
                    self.bankniftyupdater.run()
                    # self.niftyupdater.run()
                except Exception as e:
                    print(f"Error during update: {str(e)}")
                time.sleep(self.banknifty_interval)
        except KeyboardInterrupt:
            print("Stopped by user.")
        except Exception as e:
            print("Unexpected error:", e)

integrator = DashboardIntegrator(banknifty_args, indices_args, banknifty_interval=10)
integrator.run()

Fetching Bank Nifty option chain data...
Data fetched successfully at 01-Aug-2025 15:30:00. Preparing DataFrame...
DataFrame prepared. Writing to Excel...
Error during update: (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2146777998), None)
Error during update: (-2146777998, 'OLE error 0x800ac472', None, None)
Fetching Bank Nifty option chain data...
Data fetched successfully at 01-Aug-2025 15:30:00. Preparing DataFrame...
DataFrame prepared. Writing to Excel...
AutoFilter error: (-2147352567, 'Exception occurred.', (0, 'Microsoft Excel', 'AutoFilter method of Range class failed', 'xlmain11.chm', 0, -2146827284), None)
Bank Nifty option chain data updated successfully at 01-Aug-2025 15:30:00.
Fetching Bank Nifty option chain data...
Data fetched successfully at 01-Aug-2025 15:30:00. Preparing DataFrame...
DataFrame prepared. Writing to Excel...
Bank Nifty option chain data updated successfully at 01-Aug-2025 15:30:00.
Stopped by user.


# Starting to check for 2 worksheets

In [60]:
import requests
import random
import pandas as pd
import os
from pathlib import Path
import win32com.client as win32
import win32com
import time
from datetime import datetime, timedelta
import pyotp
import pandas as pd
from SmartApi import SmartConnect
from logzero import logger
import requests
import pythoncom

class OptionChainUpdaterCOM:
    def __init__(self, file_path: str):
        self.file_path = Path(file_path)
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
            "Accept": "application/json, text/plain, */*",
            "Referer": "https://www.nseindia.com/option-chain"
        }

    def fetch_data(self):
        random_suffix = random.randint(100000, 999999)
        oc_url = f"https://www.nseindia.com/api/option-chain-indices?symbol=NIFTY&c={random_suffix}"

        session = requests.Session()
        session.get("https://www.nseindia.com/option-chain", headers=self.headers)
        response = session.get(oc_url, headers=self.headers)

        if response.status_code != 200:
            raise Exception(f"Failed to fetch data. Status code: {response.status_code}")

        full_data = response.json()
        return full_data['records']['data'], full_data['records']['expiryDates'][:1], full_data['records']['timestamp']

    def prepare_dataframe(self, data, top_expiries):
        rows = []
        for item in data:
            expiry = item.get("expiryDate")
            if expiry not in top_expiries:
                continue

            row = {
                "EXPIRY DATE": expiry,
                "STRIKE": item.get("strikePrice")
            }

            ce = item.get("CE", {})
            row.update({
                "CE OI": ce.get("openInterest"),
                "CE CHNG IN OI": ce.get("changeinOpenInterest"),
                "CE VOLUME": ce.get("totalTradedVolume"),
                "CE IV": ce.get("impliedVolatility"),
                "CE LTP": ce.get("lastPrice"),
                "CE CHNG": ce.get("change"),
                "CE BID QTY": ce.get("bidQty"),
                "CE BID": ce.get("bidprice"),
                "CE ASK": ce.get("askPrice"),
                "CE ASK QTY": ce.get("askQty"),
                "CE TOTAL BUY": ce.get("totalBuyQuantity"),
                "CE TOTAL SELL": ce.get("totalSellQuantity"),
            })

            pe = item.get("PE", {})
            row.update({
                "PE OI": pe.get("openInterest"),
                "PE CHNG IN OI": pe.get("changeinOpenInterest"),
                "PE VOLUME": pe.get("totalTradedVolume"),
                "PE IV": pe.get("impliedVolatility"),
                "PE LTP": pe.get("lastPrice"),
                "PE CHNG": pe.get("change"),
                "PE BID QTY": pe.get("bidQty"),
                "PE BID": pe.get("bidprice"),
                "PE ASK": pe.get("askPrice"),
                "PE ASK QTY": pe.get("askQty"),
                "PE TOTAL BUY": pe.get("totalBuyQuantity"),
                "PE TOTAL SELL": pe.get("totalSellQuantity"),
            })

            rows.append(row)

        df = pd.DataFrame(rows)
        df.sort_values(by=["EXPIRY DATE", "STRIKE"], inplace=True)
        return df

    def write_using_com(self, df: pd.DataFrame, timestamp: str):
        def safe_set_attr(obj_getter, attr, value, retries=5, delay=0.5):
            for i in range(retries):
                try:
                    obj = obj_getter()
                    setattr(obj, attr, value)
                    return
                except Exception as e:
                    if '0x800ac472' in str(e):
                        time.sleep(delay)
                    else:
                        raise
            print(f"Failed to set {attr} after {retries} attempts.")

        pythoncom.CoInitialize()
        excel = win32.Dispatch("Excel.Application")
        excel.Visible = True
        excel.DisplayAlerts = False

        try:
            # Check if workbook is already open
            wb = None
            for open_wb in excel.Workbooks:
                if open_wb.FullName.lower() == str(self.file_path).lower():
                    wb = open_wb
                    break

            if wb is None:
                wb = excel.Workbooks.Open(str(self.file_path))

            if wb.Sheets.Count < 1:
                wb.Sheets.Add(After=wb.Sheets(wb.Sheets.Count))  # Add new sheet after the last one

            # Use the second worksheet
            ws = wb.Worksheets(1)
            ws.Name = "Nifty option chain"

            try:
                if ws.ProtectContents:
                    ws.Unprotect()
            except Exception as e:
                print(f"Warning: Couldn't unprotect sheet: {e}")

            # Clear the worksheet contents
            ws.Cells.Clear()

            safe_set_attr(lambda: ws.Cells(1, 1), "Value", f"Data fetched at: {timestamp}")

            headers = df.columns.tolist()
            num_general = 3
            num_calls = 10
            num_puts = len(headers) - num_general - num_calls

            for i in range(num_general):
                cell = ws.Cells(2, i + 1)
                safe_set_attr(lambda c=cell: c, "Value", "")
                safe_set_attr(lambda c=cell: c.Font, "Bold", True)
                safe_set_attr(lambda c=cell: c.Interior, "Color", 65535)

            if num_calls > 0:
                ws.Range(
                    ws.Cells(2, num_general + 1),
                    ws.Cells(2, num_general + num_calls)
                ).Merge()
                call_cell = ws.Cells(2, num_general + 1)
                safe_set_attr(lambda: call_cell, "Value", "CALLS")
                safe_set_attr(lambda: call_cell, "HorizontalAlignment", -4108)
                safe_set_attr(lambda: call_cell.Font, "Bold", True)
                safe_set_attr(lambda: call_cell.Interior, "Color", 65535)

            if num_puts > 0:
                ws.Range(
                    ws.Cells(2, num_general + num_calls + 1),
                    ws.Cells(2, num_general + num_calls + num_puts)
                ).Merge()
                put_cell = ws.Cells(2, num_general + num_calls + 1)
                safe_set_attr(lambda: put_cell, "Value", "PUTS")
                safe_set_attr(lambda: put_cell, "HorizontalAlignment", -4108)
                safe_set_attr(lambda: put_cell.Font, "Bold", True)
                safe_set_attr(lambda: put_cell.Interior, "Color", 65535)

            for col_index, header in enumerate(headers, start=1):
                cell = ws.Cells(3, col_index)
                safe_set_attr(lambda c=cell: c, "Value", header)
                safe_set_attr(lambda c=cell: c.Font, "Bold", True)

            for row_index, row in enumerate(df.itertuples(index=False), start=4):
                for col_index, value in enumerate(row, start=1):
                    safe_set_attr(lambda r=row_index, c=col_index: ws.Cells(r, c), "Value", value)

            last_col = df.shape[1]
            last_row = df.shape[0] + 3

            try:
                if df.shape[0] > 0 and df.shape[1] > 0 and last_row >= 3 and last_col >= 1:
                    ws.Range(ws.Cells(3, 1), ws.Cells(last_row, last_col)).AutoFilter()
                else:
                    print("No data to filter, skipping AutoFilter.")
            except Exception as e:
                print(f"AutoFilter error: {e}")

            wb.Save()

        except Exception as e:
            print(f"Error during update: {e}")

        finally:
            # Restore Excel settings
            excel.DisplayAlerts = True
            pythoncom.CoUninitialize()
            print(f"Nifty option chain data updated successfully at {timestamp}.")

    def run(self):
        print("Fetching Nifty option chain data...")
        data, expiries, timestamp = self.fetch_data()
        print(f"Data fetched at: {timestamp}")
        df = self.prepare_dataframe(data, expiries)
        print("Preparing DataFrame...")
        self.write_using_com(df, timestamp)
        print(f"Nifty options updated in Excel at {timestamp}")

class BankOptionChainUpdaterCOM:
    def __init__(self, file_path: str):
        self.file_path = Path(file_path)
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
            "Accept": "application/json, text/plain, */*",
            "Referer": "https://www.nseindia.com/option-chain"
        }

    def fetch_data(self):
        random_suffix = random.randint(100000, 999999)
        oc_url = f"https://www.nseindia.com/api/option-chain-indices?symbol=BANKNIFTY&c={random_suffix}"

        session = requests.Session()
        session.get("https://www.nseindia.com/option-chain", headers=self.headers)
        response = session.get(oc_url, headers=self.headers)

        if response.status_code != 200:
            raise Exception(f"Failed to fetch data. Status code: {response.status_code}")

        full_data = response.json()
        return full_data['records']['data'], full_data['records']['expiryDates'][:1], full_data['records']['timestamp']

    def prepare_dataframe(self, data, top_expiries):
        rows = []
        for item in data:
            expiry = item.get("expiryDate")
            if expiry not in top_expiries:
                continue

            row = {
                "EXPIRY DATE": expiry,
                "STRIKE": item.get("strikePrice")
            }

            ce = item.get("CE", {})
            row.update({
                "CE OI": ce.get("openInterest"),
                "CE CHNG IN OI": ce.get("changeinOpenInterest"),
                "CE VOLUME": ce.get("totalTradedVolume"),
                "CE IV": ce.get("impliedVolatility"),
                "CE LTP": ce.get("lastPrice"),
                "CE CHNG": ce.get("change"),
                "CE BID QTY": ce.get("bidQty"),
                "CE BID": ce.get("bidprice"),
                "CE ASK": ce.get("askPrice"),
                "CE ASK QTY": ce.get("askQty"),
                "CE TOTAL BUY": ce.get("totalBuyQuantity"),
                "CE TOTAL SELL": ce.get("totalSellQuantity"),
            })

            pe = item.get("PE", {})
            row.update({
                "PE OI": pe.get("openInterest"),
                "PE CHNG IN OI": pe.get("changeinOpenInterest"),
                "PE VOLUME": pe.get("totalTradedVolume"),
                "PE IV": pe.get("impliedVolatility"),
                "PE LTP": pe.get("lastPrice"),
                "PE CHNG": pe.get("change"),
                "PE BID QTY": pe.get("bidQty"),
                "PE BID": pe.get("bidprice"),
                "PE ASK": pe.get("askPrice"),
                "PE ASK QTY": pe.get("askQty"),
                "PE TOTAL BUY": pe.get("totalBuyQuantity"),
                "PE TOTAL SELL": pe.get("totalSellQuantity"),
            })

            rows.append(row)

        df = pd.DataFrame(rows)
        df.sort_values(by=["EXPIRY DATE", "STRIKE"], inplace=True)
        return df
    
    def write_using_com(self, df: pd.DataFrame, timestamp: str):
        def safe_set_attr(obj_getter, attr, value, retries=5, delay=0.5):
            for i in range(retries):
                try:
                    obj = obj_getter()
                    setattr(obj, attr, value)
                    return
                except Exception as e:
                    if '0x800ac472' in str(e):
                        time.sleep(delay)
                    else:
                        raise
            print(f"Failed to set {attr} after {retries} attempts.")

        pythoncom.CoInitialize()
        excel = win32.Dispatch("Excel.Application")
        excel.Visible = True

        # Temporarily disable screen updating and alert
        excel.DisplayAlerts = False

        try:
            # Check if workbook is already open
            wb = None
            for open_wb in excel.Workbooks:
                if open_wb.FullName.lower() == str(self.file_path).lower():
                    wb = open_wb
                    break

            if wb is None:
                wb = excel.Workbooks.Open(str(self.file_path))

            while wb.Sheets.Count < 1:
                wb.Sheets.Add(After=wb.Sheets(wb.Sheets.Count))

            ws = wb.Worksheets(1)
            ws.Name = "Bank_Nifty option chain"

            try:
                if ws.ProtectContents:
                    ws.Unprotect()
            except Exception as e:
                print(f"Warning: Couldn't unprotect sheet: {e}")

            ws.Cells.Clear()

            safe_set_attr(lambda: ws.Cells(1, 1), "Value", f"Data fetched at: {timestamp}")

            headers = df.columns.tolist()
            num_general = 3
            num_calls = 10
            num_puts = len(headers) - num_general - num_calls

            for i in range(num_general):
                cell = ws.Cells(2, i + 1)
                safe_set_attr(lambda c=cell: c, "Value", "")
                safe_set_attr(lambda c=cell: c.Font, "Bold", True)
                safe_set_attr(lambda c=cell: c.Interior, "Color", 65535)

            if num_calls > 0:
                ws.Range(
                    ws.Cells(2, num_general + 1),
                    ws.Cells(2, num_general + num_calls)
                ).Merge()
                call_cell = ws.Cells(2, num_general + 1)
                safe_set_attr(lambda: call_cell, "Value", "CALLS")
                safe_set_attr(lambda: call_cell, "HorizontalAlignment", -4108)
                safe_set_attr(lambda: call_cell.Font, "Bold", True)
                safe_set_attr(lambda: call_cell.Interior, "Color", 65535)

            if num_puts > 0:
                ws.Range(
                    ws.Cells(2, num_general + num_calls + 1),
                    ws.Cells(2, num_general + num_calls + num_puts)
                ).Merge()
                put_cell = ws.Cells(2, num_general + num_calls + 1)
                safe_set_attr(lambda: put_cell, "Value", "PUTS")
                safe_set_attr(lambda: put_cell, "HorizontalAlignment", -4108)
                safe_set_attr(lambda: put_cell.Font, "Bold", True)
                safe_set_attr(lambda: put_cell.Interior, "Color", 65535)

            for col_index, header in enumerate(headers, start=1):
                cell = ws.Cells(3, col_index)
                safe_set_attr(lambda c=cell: c, "Value", header)
                safe_set_attr(lambda c=cell: c.Font, "Bold", True)

            for row_index, row in enumerate(df.itertuples(index=False), start=4):
                for col_index, value in enumerate(row, start=1):
                    safe_set_attr(lambda r=row_index, c=col_index: ws.Cells(r, c), "Value", value)

            last_col = df.shape[1]
            last_row = df.shape[0] + 3

            try:
                if df.shape[0] > 0 and df.shape[1] > 0 and last_row >= 3 and last_col >= 1:
                    ws.Range(ws.Cells(3, 1), ws.Cells(last_row, last_col)).AutoFilter()
                else:
                    print("No data to filter, skipping AutoFilter.")
            except Exception as e:
                print(f"AutoFilter error: {e}")

            wb.Save()

        except Exception as e:
            print(f"Error during update: {e}")

        finally:
            # Restore Excel settings
            excel.DisplayAlerts = True
            pythoncom.CoUninitialize()
            print(f"Bank Nifty option chain data updated successfully at {timestamp}.")

    def run(self):
        print("Fetching Bank Nifty option chain data...")
        data, expiries, timestamp = self.fetch_data()
        print(f"Data fetched successfully at {timestamp}. Preparing DataFrame...")
        df = self.prepare_dataframe(data, expiries)
        print("DataFrame prepared. Writing to Excel...")
        self.write_using_com(df, timestamp)

In [63]:
file_path = r"C:\Users\jaiad\Pro T\Final_draft1\interactive_trials.xls"
file_path1 = r"C:\Users\jaiad\Pro T\Final_draft1\interactive_trials1.xls"

banknifty_args = {
    "api_key": '3tA3noq3',
    "username": 'AAAF620196',
    "pwd": '0054',
    "token": 'YS4MOR7OUZEHGNV3VFOQ5TVMRE',
    "file_path": file_path,
    "offset_rows": 15
}
indices_args = {
    "api_key": '3tA3noq3',
    "username": 'AAAF620196',
    "pwd": '0054',
    "token": 'YS4MOR7OUZEHGNV3VFOQ5TVMRE',
    "file_path": file_path
}


class DashboardIntegrator:
    def __init__(
        self,
        banknifty_args,
        indices_args,
        banknifty_interval=30
    ):
        # self.banknifty_dashboard = BankNiftyDashboard(**banknifty_args)
        # self.indices_dashboard = IndicesDashboard(**indices_args)
        # self.indices_dashboard.clear_dashboard()
        # self.indices_dashboard.setup_excel_structure()
        self.banknifty_interval = banknifty_interval
        self.niftyupdater = OptionChainUpdaterCOM(file_path=file_path1)
        self.bankniftyupdater = BankOptionChainUpdaterCOM(file_path=file_path)

    def run(self):

        try:
            while True:
                try:
                    # print("Updating Indices Dashboard...")
                    # df = self.indices_dashboard.fetch_and_process_data()
                    # df = self.indices_dashboard.calculate_sentiment(df)
                    # self.indices_dashboard.update_excel(df)
                    # print("Updating BankNifty Dashboard...")
                    # self.banknifty_dashboard.update_excel()
                    # print(f"Both dashboards updated at {datetime.now().strftime('%H:%M:%S')}")
                    self.bankniftyupdater.run()
                    self.niftyupdater.run()
                except Exception as e:
                    print(f"Error during update: {str(e)}")
                time.sleep(self.banknifty_interval)
        except KeyboardInterrupt:
            print("Stopped by user.")
        except Exception as e:
            print("Unexpected error:", e)

integrator = DashboardIntegrator(banknifty_args, indices_args, banknifty_interval=10)
integrator.run()

Fetching Bank Nifty option chain data...
Data fetched successfully at 01-Aug-2025 15:30:00. Preparing DataFrame...
DataFrame prepared. Writing to Excel...
AutoFilter error: (-2147352567, 'Exception occurred.', (0, 'Microsoft Excel', 'AutoFilter method of Range class failed', 'xlmain11.chm', 0, -2146827284), None)
Bank Nifty option chain data updated successfully at 01-Aug-2025 15:30:00.
Fetching Nifty option chain data...
Data fetched at: 01-Aug-2025 15:30:00
Preparing DataFrame...
Error during update: (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2146777998), None)
Error during update: (-2146777998, 'OLE error 0x800ac472', None, None)
Fetching Bank Nifty option chain data...
Data fetched successfully at 01-Aug-2025 15:30:00. Preparing DataFrame...
DataFrame prepared. Writing to Excel...
Error during update: (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2146777998), None)
Error during update: (-2146777998, 'OLE error 0x800ac472', None, None)
Fetching B

# option Chain final OK

In [2]:
import requests
import random
import pandas as pd
import os
from pathlib import Path
import win32com.client as win32
import win32com
import time
from datetime import datetime, timedelta
import pyotp
import pandas as pd
from SmartApi import SmartConnect
from logzero import logger
import requests
import pythoncom

class OptionChainUpdaterCOM:
    def __init__(self, file_path: str):
        self.file_path = Path(file_path)
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
            "Accept": "application/json, text/plain, */*",
            "Referer": "https://www.nseindia.com/option-chain"
        }

    def fetch_data(self):
        random_suffix = random.randint(100000, 999999)
        oc_url = f"https://www.nseindia.com/api/option-chain-indices?symbol=NIFTY&c={random_suffix}"

        session = requests.Session()
        session.get("https://www.nseindia.com/option-chain", headers=self.headers)
        response = session.get(oc_url, headers=self.headers)

        if response.status_code != 200:
            raise Exception(f"Failed to fetch data. Status code: {response.status_code}")

        full_data = response.json()
        return full_data['records']['data'], full_data['records']['expiryDates'][:1], full_data['records']['timestamp']

    def prepare_dataframe(self, data, top_expiries):
        rows = []
        for item in data:
            expiry = item.get("expiryDate")
            if expiry not in top_expiries:
                continue

            row = {
                "EXPIRY DATE": expiry,
                "STRIKE": item.get("strikePrice")
            }

            ce = item.get("CE", {})
            row.update({
                "CE OI": ce.get("openInterest"),
                "CE CHNG IN OI": ce.get("changeinOpenInterest"),
                "CE VOLUME": ce.get("totalTradedVolume"),
                "CE IV": ce.get("impliedVolatility"),
                "CE LTP": ce.get("lastPrice"),
                "CE CHNG": ce.get("change"),
                "CE BID QTY": ce.get("bidQty"),
                "CE BID": ce.get("bidprice"),
                "CE ASK": ce.get("askPrice"),
                "CE ASK QTY": ce.get("askQty"),
            })

            pe = item.get("PE", {})
            row.update({
                "PE OI": pe.get("openInterest"),
                "PE CHNG IN OI": pe.get("changeinOpenInterest"),
                "PE VOLUME": pe.get("totalTradedVolume"),
                "PE IV": pe.get("impliedVolatility"),
                "PE LTP": pe.get("lastPrice"),
                "PE CHNG": pe.get("change"),
                "PE BID QTY": pe.get("bidQty"),
                "PE BID": pe.get("bidprice"),
                "PE ASK": pe.get("askPrice"),
                "PE ASK QTY": pe.get("askQty"),
            })

            rows.append(row)

        df = pd.DataFrame(rows)
        
        # Define the desired column order based on the Excel header image
        column_order = [
            "EXPIRY DATE",
            "CE OI", "CE CHNG IN OI", "CE VOLUME", "CE IV", "CE LTP", "CE CHNG", 
            "CE BID QTY", "CE BID", "CE ASK", "CE ASK QTY",
            "STRIKE",
            "PE BID QTY", "PE BID", "PE ASK", "PE ASK QTY", "PE CHNG", "PE LTP", 
            "PE IV", "PE VOLUME", "PE CHNG IN OI", "PE OI"
        ]
        
        # Reorder the DataFrame columns
        df = df[column_order]
        
        # Sort by EXPIRY DATE and STRIKE
        df.sort_values(by=["EXPIRY DATE", "STRIKE"], inplace=True)
        return df

    def write_using_com(self, df: pd.DataFrame, timestamp: str):
        def safe_set_attr(obj_getter, attr, value, retries=5, delay=0.5):
            for i in range(retries):
                try:
                    obj = obj_getter()
                    setattr(obj, attr, value)
                    return
                except Exception as e:
                    if '0x800ac472' in str(e):
                        time.sleep(delay)
                    else:
                        raise
            print(f"Failed to set {attr} after {retries} attempts.")

        pythoncom.CoInitialize()
        excel = win32com.client.gencache.EnsureDispatch("Excel.Application")
        excel.Visible = True

        # Temporarily disable screen updating and alerts
        excel.DisplayAlerts = False

        try:
            # Check if workbook is already open
            wb = None
            for open_wb in excel.Workbooks:
                if open_wb.FullName.lower() == str(self.file_path).lower():
                    wb = open_wb
                    break

            if wb is None:
                wb = excel.Workbooks.Open(str(self.file_path))

            while wb.Sheets.Count < 1:
                wb.Sheets.Add(After=wb.Sheets(wb.Sheets.Count))

            ws = wb.Worksheets(1)
            ws.Name = "Nifty option chain"

            try:
                if ws.ProtectContents:
                    ws.Unprotect()
            except Exception as e:
                print(f"Warning: Couldn't unprotect sheet: {e}")

            ws.Cells.Clear()

            safe_set_attr(lambda: ws.Cells(1, 1), "Value", f"Data fetched at: {timestamp}")

            headers = df.columns.tolist()
            num_general = 1  # Only EXPIRY DATE
            num_calls = 10   # CE columns
            num_puts = 10    # PE columns

            # Row 2 - Group headers
            for i in range(num_general):
                cell = ws.Cells(2, i + 1)
                safe_set_attr(lambda c=cell: c, "Value", "")
                safe_set_attr(lambda c=cell: c.Font, "Bold", True)
                safe_set_attr(lambda c=cell: c.Interior, "Color", 65535)

            if num_calls > 0:
                ws.Range(
                    ws.Cells(2, num_general + 1),
                    ws.Cells(2, num_general + num_calls)
                ).Merge()
                call_cell = ws.Cells(2, num_general + 1)
                safe_set_attr(lambda: call_cell, "Value", "CALLS")
                safe_set_attr(lambda: call_cell, "HorizontalAlignment", -4108)
                safe_set_attr(lambda: call_cell.Font, "Bold", True)
                safe_set_attr(lambda: call_cell.Interior, "Color", 65535)

            # STRIKE header
            strike_cell = ws.Cells(2, num_general + num_calls + 1)
            safe_set_attr(lambda c=strike_cell: c, "Value", "STRIKE")
            safe_set_attr(lambda c=strike_cell: c.Font, "Bold", True)
            safe_set_attr(lambda c=strike_cell: c.Interior, "Color", 65535)

            if num_puts > 0:
                ws.Range(
                    ws.Cells(2, num_general + num_calls + 2),
                    ws.Cells(2, num_general + num_calls + 1 + num_puts)
                ).Merge()
                put_cell = ws.Cells(2, num_general + num_calls + 2)
                safe_set_attr(lambda: put_cell, "Value", "PUTS")
                safe_set_attr(lambda: put_cell, "HorizontalAlignment", -4108)
                safe_set_attr(lambda: put_cell.Font, "Bold", True)
                safe_set_attr(lambda: put_cell.Interior, "Color", 65535)

            # Row 3 - Actual headers
            for col_index, header in enumerate(headers, start=1):
                cell = ws.Cells(3, col_index)
                safe_set_attr(lambda c=cell: c, "Value", header)
                safe_set_attr(lambda c=cell: c.Font, "Bold", True)

            # Write data
            for row_index, row in enumerate(df.itertuples(index=False), start=4):
                for col_index, value in enumerate(row, start=1):
                    safe_set_attr(lambda r=row_index, c=col_index: ws.Cells(r, c), "Value", value)

            last_col = df.shape[1]
            last_row = df.shape[0] + 3

            try:
                if df.shape[0] > 0 and df.shape[1] > 0 and last_row >= 3 and last_col >= 1:
                    ws.Range(ws.Cells(3, 1), ws.Cells(last_row, last_col)).AutoFilter()
                else:
                    print("No data to filter, skipping AutoFilter.")
            except Exception as e:
                print(f"AutoFilter error: {e}")

        except Exception as e:
            print(f"Error during update: {e}")

        finally:
            # Restore Excel settings
            excel.DisplayAlerts = True
            pythoncom.CoUninitialize()
            print(f"Nifty option chain data updated successfully at {timestamp}.")

    def run(self):
        print("Fetching Nifty option chain data...")
        data, expiries, timestamp = self.fetch_data()
        print(f"Data fetched at: {timestamp}")
        df = self.prepare_dataframe(data, expiries)
        print("Preparing DataFrame...")
        self.write_using_com(df, timestamp)
        print(f"Nifty options updated in Excel at {timestamp}")

class BankOptionChainUpdaterCOM:
    def __init__(self, file_path: str):
        self.file_path = Path(file_path)
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
            "Accept": "application/json, text/plain, */*",
            "Referer": "https://www.nseindia.com/option-chain"
        }

    def fetch_data(self):
        random_suffix = random.randint(100000, 999999)
        oc_url = f"https://www.nseindia.com/api/option-chain-indices?symbol=BANKNIFTY&c={random_suffix}"

        session = requests.Session()
        session.get("https://www.nseindia.com/option-chain", headers=self.headers)
        response = session.get(oc_url, headers=self.headers)

        if response.status_code != 200:
            raise Exception(f"Failed to fetch data. Status code: {response.status_code}")

        full_data = response.json()
        return full_data['records']['data'], full_data['records']['expiryDates'][:1], full_data['records']['timestamp']

    def prepare_dataframe(self, data, top_expiries):
        rows = []
        for item in data:
            expiry = item.get("expiryDate")
            if expiry not in top_expiries:
                continue

            row = {
                "EXPIRY DATE": expiry,
                "STRIKE": item.get("strikePrice")
            }

            ce = item.get("CE", {})
            row.update({
                "CE OI": ce.get("openInterest"),
                "CE CHNG IN OI": ce.get("changeinOpenInterest"),
                "CE VOLUME": ce.get("totalTradedVolume"),
                "CE IV": ce.get("impliedVolatility"),
                "CE LTP": ce.get("lastPrice"),
                "CE CHNG": ce.get("change"),
                "CE BID QTY": ce.get("bidQty"),
                "CE BID": ce.get("bidprice"),
                "CE ASK": ce.get("askPrice"),
                "CE ASK QTY": ce.get("askQty"),
            })

            pe = item.get("PE", {})
            row.update({
                "PE OI": pe.get("openInterest"),
                "PE CHNG IN OI": pe.get("changeinOpenInterest"),
                "PE VOLUME": pe.get("totalTradedVolume"),
                "PE IV": pe.get("impliedVolatility"),
                "PE LTP": pe.get("lastPrice"),
                "PE CHNG": pe.get("change"),
                "PE BID QTY": pe.get("bidQty"),
                "PE BID": pe.get("bidprice"),
                "PE ASK": pe.get("askPrice"),
                "PE ASK QTY": pe.get("askQty"),
            })

            rows.append(row)

        df = pd.DataFrame(rows)
        
        # Define the desired column order based on the Excel header image
        column_order = [
            "EXPIRY DATE",
            "CE OI", "CE CHNG IN OI", "CE VOLUME", "CE IV", "CE LTP", "CE CHNG", 
            "CE BID QTY", "CE BID", "CE ASK", "CE ASK QTY",
            "STRIKE",
            "PE BID QTY", "PE BID", "PE ASK", "PE ASK QTY", "PE CHNG", "PE LTP", 
            "PE IV", "PE VOLUME", "PE CHNG IN OI", "PE OI"
        ]
        
        # Reorder the DataFrame columns
        df = df[column_order]
        
        # Sort by EXPIRY DATE and STRIKE
        df.sort_values(by=["EXPIRY DATE", "STRIKE"], inplace=True)
        return df
    
    def write_using_com(self, df: pd.DataFrame, timestamp: str):
        def safe_set_attr(obj_getter, attr, value, retries=5, delay=0.5):
            for i in range(retries):
                try:
                    obj = obj_getter()
                    setattr(obj, attr, value)
                    return
                except Exception as e:
                    if '0x800ac472' in str(e):
                        time.sleep(delay)
                    else:
                        raise
            print(f"Failed to set {attr} after {retries} attempts.")

        pythoncom.CoInitialize()
        excel = win32com.client.gencache.EnsureDispatch("Excel.Application")
        excel.Visible = True

        # Temporarily disable screen updating and alerts
        excel.DisplayAlerts = False

        try:
            # Check if workbook is already open
            wb = None
            for open_wb in excel.Workbooks:
                if open_wb.FullName.lower() == str(self.file_path).lower():
                    wb = open_wb
                    break

            if wb is None:
                wb = excel.Workbooks.Open(str(self.file_path))

            while wb.Sheets.Count < 1:
                wb.Sheets.Add(After=wb.Sheets(wb.Sheets.Count))

            ws = wb.Worksheets(1)
            ws.Name = "Bank_Nifty option chain"

            try:
                if ws.ProtectContents:
                    ws.Unprotect()
            except Exception as e:
                print(f"Warning: Couldn't unprotect sheet: {e}")

            ws.Cells.Clear()

            safe_set_attr(lambda: ws.Cells(1, 1), "Value", f"Data fetched at: {timestamp}")

            headers = df.columns.tolist()
            num_general = 1  # Only EXPIRY DATE
            num_calls = 10   # CE columns
            num_puts = 10    # PE columns

            # Row 2 - Group headers
            for i in range(num_general):
                cell = ws.Cells(2, i + 1)
                safe_set_attr(lambda c=cell: c, "Value", "")
                safe_set_attr(lambda c=cell: c.Font, "Bold", True)
                safe_set_attr(lambda c=cell: c.Interior, "Color", 65535)

            if num_calls > 0:
                ws.Range(
                    ws.Cells(2, num_general + 1),
                    ws.Cells(2, num_general + num_calls)
                ).Merge()
                call_cell = ws.Cells(2, num_general + 1)
                safe_set_attr(lambda: call_cell, "Value", "CALLS")
                safe_set_attr(lambda: call_cell, "HorizontalAlignment", -4108)
                safe_set_attr(lambda: call_cell.Font, "Bold", True)
                safe_set_attr(lambda: call_cell.Interior, "Color", 65535)

            # STRIKE header
            strike_cell = ws.Cells(2, num_general + num_calls + 1)
            safe_set_attr(lambda c=strike_cell: c, "Value", "STRIKE")
            safe_set_attr(lambda c=strike_cell: c.Font, "Bold", True)
            safe_set_attr(lambda c=strike_cell: c.Interior, "Color", 65535)

            if num_puts > 0:
                ws.Range(
                    ws.Cells(2, num_general + num_calls + 2),
                    ws.Cells(2, num_general + num_calls + 1 + num_puts)
                ).Merge()
                put_cell = ws.Cells(2, num_general + num_calls + 2)
                safe_set_attr(lambda: put_cell, "Value", "PUTS")
                safe_set_attr(lambda: put_cell, "HorizontalAlignment", -4108)
                safe_set_attr(lambda: put_cell.Font, "Bold", True)
                safe_set_attr(lambda: put_cell.Interior, "Color", 65535)

            # Row 3 - Actual headers
            for col_index, header in enumerate(headers, start=1):
                cell = ws.Cells(3, col_index)
                safe_set_attr(lambda c=cell: c, "Value", header)
                safe_set_attr(lambda c=cell: c.Font, "Bold", True)

            # Write data
            for row_index, row in enumerate(df.itertuples(index=False), start=4):
                for col_index, value in enumerate(row, start=1):
                    safe_set_attr(lambda r=row_index, c=col_index: ws.Cells(r, c), "Value", value)

            last_col = df.shape[1]
            last_row = df.shape[0] + 3

            try:
                if df.shape[0] > 0 and df.shape[1] > 0 and last_row >= 3 and last_col >= 1:
                    ws.Range(ws.Cells(3, 1), ws.Cells(last_row, last_col)).AutoFilter()
                else:
                    print("No data to filter, skipping AutoFilter.")
            except Exception as e:
                print(f"AutoFilter error: {e}")

        except Exception as e:
            print(f"Error during update: {e}")

        finally:
            # Restore Excel settings
            excel.DisplayAlerts = True
            pythoncom.CoUninitialize()
            print(f"Bank Nifty option chain data updated successfully at {timestamp}.")

    def run(self):
        print("Fetching Bank Nifty option chain data...")
        data, expiries, timestamp = self.fetch_data()
        print(f"Data fetched successfully at {timestamp}. Preparing DataFrame...")
        df = self.prepare_dataframe(data, expiries)
        print("DataFrame prepared. Writing to Excel...")
        self.write_using_com(df, timestamp)

Rebuilding cache of generated files for COM support...
Checking 00020813-0000-0000-C000-000000000046x0x1x9
Could not add module (IID('{00020813-0000-0000-C000-000000000046}'), 0, 1, 9) - <class 'AttributeError'>: module 'win32com.gen_py.00020813-0000-0000-C000-000000000046x0x1x9' has no attribute 'CLSIDToClassMap'
Done.


[E 250828 21:03:35 smartConnect:78] Exception while retriving IP Address,using local host IP address: HTTPSConnectionPool(host='api.ipify.org', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000226C5F1D760>: Failed to resolve 'api.ipify.org' ([Errno 11001] getaddrinfo failed)"))


In [5]:
file_path = r"C:\Users\jaiad\Pro T\Final_draft1\interactive_trials.xls"
file_path1 = r"C:\Users\jaiad\Pro T\Final_draft1\interactive_trials1.xls"

banknifty_args = {
    "api_key": '3tA3noq3',
    "username": 'AAAF620196',
    "pwd": '0054',
    "token": 'YS4MOR7OUZEHGNV3VFOQ5TVMRE',
    "file_path": file_path,
    "offset_rows": 15
}
indices_args = {
    "api_key": '3tA3noq3',
    "username": 'AAAF620196',
    "pwd": '0054',
    "token": 'YS4MOR7OUZEHGNV3VFOQ5TVMRE',
    "file_path": file_path
}


class DashboardIntegrator:
    def __init__(
        self,
        banknifty_args,
        indices_args,
        banknifty_interval=30
    ):
        # self.banknifty_dashboard = BankNiftyDashboard(**banknifty_args)
        # self.indices_dashboard = IndicesDashboard(**indices_args)
        # self.indices_dashboard.clear_dashboard()
        # self.indices_dashboard.setup_excel_structure()
        self.banknifty_interval = banknifty_interval
        self.niftyupdater = OptionChainUpdaterCOM(file_path=file_path1)
        self.bankniftyupdater = BankOptionChainUpdaterCOM(file_path=file_path)

    def run(self):

        try:
            while True:
                try:
                    # print("Updating Indices Dashboard...")
                    # df = self.indices_dashboard.fetch_and_process_data()
                    # df = self.indices_dashboard.calculate_sentiment(df)
                    # self.indices_dashboard.update_excel(df)
                    # print("Updating BankNifty Dashboard...")
                    # self.banknifty_dashboard.update_excel()
                    # print(f"Both dashboards updated at {datetime.now().strftime('%H:%M:%S')}")
                    self.niftyupdater.run()
                    self.bankniftyupdater.run()
                except Exception as e:
                    print(f"Error during update: {str(e)}")
                time.sleep(self.banknifty_interval)
        except KeyboardInterrupt:
            print("Stopped by user.")
        except Exception as e:
            print("Unexpected error:", e)

integrator = DashboardIntegrator(banknifty_args, indices_args, banknifty_interval=10)
integrator.run()

Fetching Nifty option chain data...
Data fetched at: 28-Aug-2025 15:30:00
Preparing DataFrame...
Nifty option chain data updated successfully at 28-Aug-2025 15:30:00.
Nifty options updated in Excel at 28-Aug-2025 15:30:00
Fetching Bank Nifty option chain data...
Data fetched successfully at 28-Aug-2025 15:30:00. Preparing DataFrame...
DataFrame prepared. Writing to Excel...
Bank Nifty option chain data updated successfully at 28-Aug-2025 15:30:00.
Stopped by user.
